<p style="text-align: center; font-size:24px;"><b>Data Cleaning</b></p>

A major part of data science is not building models but preparing data so that those models can work properly. In fact, most of the effort goes into cleaning and organizing raw information before any analysis takes place. Raw data often arrives messy, with inconsistencies and formats that make it hard to use directly. To make it usable, data scientists diagnose how “tidy” it is, reshape rows and columns so they line up with the questions being asked, and sometimes merge multiple sources together. They may need to correct value types, such as converting numbers that were stored as text, or handle gaps by dropping or filling missing entries. Another frequent task is manipulating strings to make the data more consistent and meaningful. All these steps are essential because clean, well-structured data is the foundation for reliable insights. Through practice on untidy datasets, one learns the key techniques that transform raw information into a state ready for exploration and analysis.


In [1]:
import pandas as pd

# Diagnose the data
Tidy data is the standard we aim for when preparing information for analysis. In tidy form, every column represents one variable and every row represents one observation. This structure makes it easy to filter, aggregate, and visualize results. When data isn’t tidy, it often has values spread across columns or compressed into a single field, which makes analysis clumsy. For instance, a wide table with separate columns for “Checkings” and “Savings” balances is harder to work with than a long-form table where account type is a variable and each balance has its own row. By reshaping into the long form, you make the structure consistent and flexible.
Before reshaping or cleaning, the first step is diagnosing the state of your dataset. With pandas, you can quickly get a sense of its shape and issues: `.head()` shows the first rows to check for structure, `.info()` summarizes data types and missing values, `.describe()` provides statistical overviews, `.columns` lists the headers, and `.value_counts()` reveals unique values in a column. These functions help you detect untidy patterns, inconsistencies, or errors so you can plan the cleaning steps needed to transform the data into a tidy, usable format.

In [2]:
# Read the CSV files from the new path "datafolder/"
df1 = pd.read_csv("data_folder/df1.csv")   # Load df1.csv from the datafolder directory
df2 = pd.read_csv("data_folder/df2.csv")   # Load df2.csv from the datafolder directory

# Display the first 5 rows of each DataFrame
print(df1.head())  # Quick check of df1 structure
print(df2.head())  # Quick check of df2 structure

# Display info summary (columns, data types, non-null counts)
print(df1.info())  # Diagnose df1
print(df2.info())  # Diagnose df2

# Display descriptive statistics (mean, std, min, max, etc.)
print(df1.describe())  # Summary stats for df1
print(df2.describe())  # Summary stats for df2

# Display distinct rows and their frequencies
print(df1.value_counts())  # Value counts for unique rows in df1
print(df2.value_counts())  # Value counts for unique rows in df2

  Grocery Item  Cake Recipe  Pancake Recipe  Cookie Recipe
0         Eggs            2               3              1
1         Milk            1               2              1
2        Flour            2               1              2
  Grocery Item          Recipe  Number
0         Eggs     Cake Recipe       2
1         Milk     Cake Recipe       1
2        Flour     Cake Recipe       2
3         Eggs  Pancake Recipe       3
4         Milk  Pancake Recipe       2
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Grocery Item    3 non-null      object
 1   Cake Recipe     3 non-null      int64 
 2   Pancake Recipe  3 non-null      int64 
 3   Cookie Recipe   3 non-null      int64 
dtypes: int64(3), object(1)
memory usage: 224.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
 #   C

# Dealing with multiples files
When working in real-world projects, data often comes split into many separate files that share the same structure. An efficient way to handle this situation is to combine them into a single dataset so that analysis can be done on the whole rather than on fragments. For example, imagine you have files named `file1.csv`, `file2.csv`, `file3.csv`, and so on. Rather than reading them one by one and joining them manually, you can use the `glob` library to automatically collect all the filenames that follow a certain pattern.
By writing `glob.glob("file*.csv")`, Python returns a list of all files in the directory whose names start with “file” and end in `.csv`. You can then loop over this list, use `pandas.read_csv()` to read each file into a DataFrame, and store each DataFrame in a list. Finally, `pd.concat()` merges all those DataFrames into a single table.
This approach means you don’t have to hard-code every filename; you just tell Python what the pattern looks like, and it handles the rest. The result is one combined DataFrame ready for cleaning, exploration, and analysis, no matter how many files you start with.

In [3]:
import pandas as pd      # pandas is used to work with tabular data
import glob              # glob is used to find filenames matching a pattern

# student_files is a list of all filenames that match "exams*.csv"
# Example: ["exams0.csv", "exams1.csv", ..., "exams9.csv"]
student_files = glob.glob("data_folder/exams*.csv")

# df_list is an empty Python list that will store multiple DataFrames
df_list = []

# Loop through each filename in student_files
for filename in student_files:
    # df is a pandas DataFrame created by reading the CSV file
    df = pd.read_csv(filename)
    # Append each DataFrame to df_list
    df_list.append(df)

# students is a single pandas DataFrame created by concatenating
# all the DataFrames stored inside df_list
students = pd.concat(df_list)

# Print the combined DataFrame
print(students)

# Print the number of rows (observations) in the combined DataFrame
print(len(students))

    id         full_name gender_age fractions probability       grade
0    0  Nikolia Rainsdon        F15       66%         74%  11th grade
1    1    Libbie MacIver        F15       71%         83%  10th grade
2    2    Caesar Arnison        M16       70%         78%  12th grade
3    3     Leanora Cowup        F15       72%         74%   9th grade
4    4     Leanora Cowup        F15       72%         74%   9th grade
..  ..               ...        ...       ...         ...         ...
95  95  Brig Meadowcroft        M16       84%         NaN  12th grade
96  96  Isidor Abrashkov        M14       83%         78%  11th grade
97  97      Siana McKune        F17       72%         76%  12th grade
98  98       Tuck Lyford        M17       73%         84%  11th grade
99  99       Janie Paris        F17       68%         82%  11th grade

[600 rows x 6 columns]
600


# Reshaping your Data
Reshaping data is often necessary to turn a dataset into tidy form, where every variable has its own column and every observation has its own row. A common issue is when different categories of the same variable are stored as separate columns. For instance, you might see account balances spread across “Checking” and “Savings” columns, which makes comparisons and aggregations awkward. The tidy version has a single “Account Type” column that indicates whether a row is a checking or savings balance, and a single “Amount” column that records the balance.
The tool for this transformation in pandas is `.melt()`. It “unpivots” wide data into long format. You specify which column(s) should remain fixed (`id_vars`), which columns you want to transform into variable/value pairs (`value_vars`), what to call the new variable column (`var_name`), and what to call the new values column (`value_name`). For example, calling `.melt()` on the account table with “Account” as the ID and “Checking” and “Savings” as the value variables produces a long table where each row is one balance, clearly identified by account number and account type.
This tidy structure is far more flexible. It makes it straightforward to group by account type, calculate totals, or merge with other datasets. After melting, it’s also common to rename the columns so they are descriptive, ensuring that the table is self-explanatory to anyone who uses it.


In [4]:
students = pd.read_csv("data_folder/students.csv")

# 1. Print the column names of the original students DataFrame
print(students.columns)   # This shows what columns exist before reshaping

# 2. Reshape the DataFrame with pd.melt()
# - Keep 'full_name', 'gender_age', and 'grade' as identifier columns
# - Melt the exam score columns: 'fractions' and 'probability'
# - Create a new column 'exam' that stores the exam type
# - Create a new column 'score' that stores the actual exam score
students = pd.melt(
    frame=students,
    id_vars=['full_name', 'gender_age', 'grade'],
    value_vars=['fractions', 'probability'],
    value_name='score',
    var_name='exam'
)
print("-----------------------------------------------------")
# 3. Print out the first rows to verify reshaping worked
print(students.head())
print("-----------------------------------------------------")
# 4. Print the new column names
print(students.columns)
print("-----------------------------------------------------")
# 5. Print the counts of each exam type
print(students['exam'].value_counts())


Index(['Unnamed: 0', 'full_name', 'gender_age', 'fractions', 'probability',
       'grade'],
      dtype='object')
-----------------------------------------------------
           full_name gender_age       grade       exam score
0     Moses Kirckman        M14  11th grade  fractions   69%
1    Timofei Strowan        M18  11th grade  fractions   63%
2       Silvain Poll        M18   9th grade  fractions   69%
3     Lezley Pinxton        M18  11th grade  fractions   NaN
4  Bernadene Saunper        F17  11th grade  fractions   72%
-----------------------------------------------------
Index(['full_name', 'gender_age', 'grade', 'exam', 'score'], dtype='object')
-----------------------------------------------------
exam
fractions      1000
probability    1000
Name: count, dtype: int64


# Dealing with Duplicates
Duplicates are a common issue in datasets, often caused by mistakes during data entry, collection, or saving. They can lead to misleading analyses if not handled properly. In pandas, you can detect duplicates with the `.duplicated()` method, which returns a series of Boolean values indicating whether each row is a repeat of a previous one. A row is considered a duplicate only if every value in that row matches another row exactly.
To clean the dataset, `.drop_duplicates()` removes all repeated rows, keeping only the first occurrence. For example, if two identical “apple” rows exist, one will be dropped while unique rows remain. However, rows that differ in even one column (like two “peach” entries with different prices) are treated as distinct and not removed.
You can also target duplicates in specific columns by using the `subset` parameter. Calling `.drop_duplicates(subset=["item"])` ensures only the first row for each item is kept, dropping any additional entries, even if other values differ. This is helpful when you know that duplicates in a certain variable (like “item”) don’t add useful information. On the other hand, you wouldn’t want to remove duplicates based on a column like “price,” because multiple items legitimately share the same price.
The key is to carefully decide which columns define uniqueness in your context. Used thoughtfully, `.duplicated()` and `.drop_duplicates()` ensure your data is clean, accurate, and reliable for analysis.

In [5]:
import pandas as pd

# Print the original DataFrame
print(students)

# 1. Create a Series showing which rows are duplicates
duplicates = students.duplicated()

# 2. Print counts of True/False → tells us how many rows are duplicates
print(duplicates.value_counts())

# 3. Drop duplicate rows and update the DataFrame
students = students.drop_duplicates()

# 4. Check again for duplicates after dropping
duplicates = students.duplicated()
print(duplicates.value_counts())


              full_name gender_age       grade         exam score
0        Moses Kirckman        M14  11th grade    fractions   69%
1       Timofei Strowan        M18  11th grade    fractions   63%
2          Silvain Poll        M18   9th grade    fractions   69%
3        Lezley Pinxton        M18  11th grade    fractions   NaN
4     Bernadene Saunper        F17  11th grade    fractions   72%
...                 ...        ...         ...          ...   ...
1995     Wilie Stillert        F14   9th grade  probability   69%
1996     Gertie Flicker        F15  11th grade  probability   86%
1997       Yettie Labes        F14  12th grade  probability   82%
1998     Lock McGuinley        M18  10th grade  probability   84%
1999       Bebe Lebbern        F15  12th grade  probability   91%

[2000 rows x 5 columns]
False    1976
True       24
Name: count, dtype: int64
False    1976
Name: count, dtype: int64


# Split Column values
Sometimes data columns contain multiple pieces of information bundled together, and we need to separate them into distinct variables to make the dataset tidy. A common case is when dates are stored as strings without delimiters, like “MMDDYYYY.” For example, “12241989” represents December 24, 1989. While this format is compact, it makes analysis harder because month, day, and year are not directly accessible.
When we know the exact structure of the string, pandas allows us to split it using string indexing. By slicing substrings with `.str`, we can create new columns. Taking the first two characters gives the month, the next two characters give the day, and the rest of the string provides the year. For instance, the value “10311966” becomes month = “10”, day = “31”, and year = “1966.”
This transformation turns one messy column into three clear variables that can be used separately in analysis or visualization. It also helps prepare the data for conversion into numeric or datetime formats later. Splitting by index is thus a quick and reliable technique whenever you have strings with a consistent fixed-length structure.

What .str does
- In pandas, a column of text values (strings) is stored as a Series.
- The .str accessor allows you to apply vectorized string operations directly to every element in the column.
- For example, .str[0] takes the first character of every string, .str[1:] takes the substring starting from the second character to the end.
- Without .str, you’d need to loop through each row manually, but .str applies the operation efficiently to the whole column.

In [6]:
# 1. Print the original DataFrame
print(students)

# Print column names to inspect structure
print(students.columns)

# 2. Inspect the first rows of the gender_age column
print(students["gender_age"].head())

# 3. Extract the first character as gender
students["gender"] = students.gender_age.str[0]

# 4. Extract the rest of the string as age
students["age"] = students.gender_age.str[1:]

# 5. Inspect the DataFrame after adding new columns
print(students.head())
print(students.columns)

# 6. Drop the old gender_age column by selecting only the needed columns
students = students[['full_name', 'grade', 'exam', 'score', 'gender', 'age']]
print(students.head())


              full_name gender_age       grade         exam score
0        Moses Kirckman        M14  11th grade    fractions   69%
1       Timofei Strowan        M18  11th grade    fractions   63%
2          Silvain Poll        M18   9th grade    fractions   69%
3        Lezley Pinxton        M18  11th grade    fractions   NaN
4     Bernadene Saunper        F17  11th grade    fractions   72%
...                 ...        ...         ...          ...   ...
1995     Wilie Stillert        F14   9th grade  probability   69%
1996     Gertie Flicker        F15  11th grade  probability   86%
1997       Yettie Labes        F14  12th grade  probability   82%
1998     Lock McGuinley        M18  10th grade  probability   84%
1999       Bebe Lebbern        F15  12th grade  probability   91%

[1976 rows x 5 columns]
Index(['full_name', 'gender_age', 'grade', 'exam', 'score'], dtype='object')
0    M14
1    M18
2    M18
3    M18
4    F17
Name: gender_age, dtype: object
           full_name gender_a

# Splitting Strings 2
Sometimes a single column contains more than one piece of information, and splitting by fixed positions won’t work if the parts have different lengths. In those cases, we can split by a specific character that separates the values. For example, if a column named type has entries like user_Kenya or admin_US, the underscore _ is the natural divider between user type and country.
With pandas, .str.split('_') separates each string into a list: everything before the underscore goes into the first position, and everything after goes into the second. By saving this intermediate result, we can then select the pieces we want. Using .str.get(0) picks the first part (e.g., user, admin), and .str.get(1) picks the second part (e.g., Kenya, US).
This process reshapes the dataset into a tidier form with two new columns, usertype and country, which makes it much easier to analyze patterns such as how many users come from each country or how behavior differs across roles.

In [7]:
# 1. Split the 'full_name' column into two parts wherever there is a space " "
name_split = students["full_name"].str.split(" ")

# 2. Take the first part (index 0) as the first name
students["first_name"] = name_split.str.get(0)

# 3. Take the second part (index 1) as the last name
students["last_name"] = name_split.str.get(1)

# 4. Print the first rows to check the new structure
print(students.head())

           full_name       grade       exam score gender age first_name  \
0     Moses Kirckman  11th grade  fractions   69%      M  14      Moses   
1    Timofei Strowan  11th grade  fractions   63%      M  18    Timofei   
2       Silvain Poll   9th grade  fractions   69%      M  18    Silvain   
3     Lezley Pinxton  11th grade  fractions   NaN      M  18     Lezley   
4  Bernadene Saunper  11th grade  fractions   72%      F  17  Bernadene   

  last_name  
0  Kirckman  
1   Strowan  
2      Poll  
3   Pinxton  
4   Saunper  


# Looking at Types
Every column in a pandas DataFrame has a data type, known as a **dtype**, and the dtype determines what kinds of operations are possible. Columns can be numeric (like `int` or `float`), boolean, dates or times (`datetime`, `timedelta`), categories, or generic objects (often strings). When data comes in from a CSV or other raw source, pandas may store some values as objects even if they look numeric, which can make analysis harder.

For example, if a column of prices is stored as strings like `"$3"`, the dtype will be `object`. That means you can’t directly calculate averages or plot trends because pandas treats the column as text rather than numbers. By contrast, a column of calorie counts stored as `int64` can be summed, averaged, or plotted without issue.

To check the types of all columns in your DataFrame, you use `.dtypes`. This returns a Series object listing the dtype of each column. For instance, in a table with fruit names, prices stored as strings, and calorie counts as integers, `.dtypes` would show `object` for the text columns and `int64` for the calorie column.

Knowing your dtypes is the first step in cleaning because it tells you which columns need conversion before analysis. The next steps often involve transforming object columns into numeric or datetime types so you can apply mathematical or time-based operations to them.


In [8]:
# 1. Inspect the data types of each column
print(students.dtypes)

# 2. Try to calculate the mean of the 'score' column
# This will cause an error if 'score' is stored as strings instead of numbers
print(students["score"].mean())

full_name     object
grade         object
exam          object
score         object
gender        object
age           object
first_name    object
last_name     object
dtype: object


TypeError: can only concatenate str (not "int") to str

# String Parsing
String parsing is an important step in cleaning data, especially when numbers are stored as text with extra characters that make them unusable for calculations. A common case is prices stored with dollar signs. While `"$3"` looks like a number, pandas reads it as a string because of the symbol, meaning you can’t average or compare values directly.
To fix this, you can use regular expressions with `.replace()` to strip out unwanted characters. For example, applying `.replace('[\$,]', '', regex=True)` to the price column removes dollar signs and commas, leaving only numeric characters in the strings. Once the column contains clean strings, you then use `pd.to_numeric()` to convert it into a numeric dtype, such as float.
After these steps, the column can be used for proper analysis: you can compute the mean price, sum totals, or make price comparisons between fruits. This transformation turns the dataset from something visually understandable but computationally awkward into a format that is both tidy and analytically useful.
By parsing strings like this, you turn inconsistent, text-based data into reliable numerical variables, a crucial step for accurate analysis and visualization.

In [9]:
# From object the score becomes float
print(students.dtypes) 

# 1. Remove the '%' signs from the score column using regex
students["score"] = students["score"].replace('[\%,]', '', regex=True)

# 2. Convert the cleaned score column from string (object) to numeric (int/float)
students["score"] = pd.to_numeric(students["score"])

# Check the result
print(students.dtypes)   # confirm 'score' is now numeric
print(students.head())   # preview the cleaned DataFrame


full_name     object
grade         object
exam          object
score         object
gender        object
age           object
first_name    object
last_name     object
dtype: object
full_name      object
grade          object
exam           object
score         float64
gender         object
age            object
first_name     object
last_name      object
dtype: object
           full_name       grade       exam  score gender age first_name  \
0     Moses Kirckman  11th grade  fractions   69.0      M  14      Moses   
1    Timofei Strowan  11th grade  fractions   63.0      M  18    Timofei   
2       Silvain Poll   9th grade  fractions   69.0      M  18    Silvain   
3     Lezley Pinxton  11th grade  fractions    NaN      M  18     Lezley   
4  Bernadene Saunper  11th grade  fractions   72.0      F  17  Bernadene   

  last_name  
0  Kirckman  
1   Strowan  
2      Poll  
3   Pinxton  
4   Saunper  


# More String Parsing
Sometimes the numbers we need for analysis are hidden inside longer text strings. For example, workout logs might store both the type of exercise and the number of repetitions in the same field: `"lunges - 30 reps"`. While this is easy for humans to read, it’s not structured enough for analysis. To make the data usable, we want to separate the exercise type from the number of reps.
With pandas, we can do this using `.str.split()` together with a regular expression. Splitting on the pattern `(\d+)` tells pandas to break the string wherever it finds digits, and because the digits are in parentheses, they are preserved as part of the result. Using `expand=True` creates a temporary DataFrame where the exercise description, number of reps, and leftover text like `"reps"` are split into separate columns.
Once we have this split DataFrame, we can assign the parts back to the original data. The middle column (the digits) becomes the `reps` column, and we convert it to numeric with `pd.to_numeric()` so we can calculate totals or averages. The first column (the exercise name) can be cleaned with `.replace('[\- ]', '', regex=True)` to remove stray hyphens or spaces, leaving just the exercise name.
The result is a tidy dataset where each row has separate, meaningful variables: `date`, `exercise`, and `reps`. Now we can easily track how many squats or lunges were done over time, compare exercises, and perform all kinds of analysis that would not be possible if everything remained trapped in a single string.

In [10]:
import pandas as pd

# 1) Prime 5 righe della colonna grade
print(students['grade'].head())

# 2) Estrai il numero con regex (più diretto di split)
students['grade'] = students['grade'].str.extract(r'(\d+)')

# 3) Tipi
print(students.dtypes)

# 4) Converti a numerico
students['grade'] = pd.to_numeric(students['grade'], errors='coerce')

# 5) Media
avg_grade = students['grade'].mean()
print(avg_grade)


0    11th grade
1    11th grade
2     9th grade
3    11th grade
4    11th grade
Name: grade, dtype: object
full_name      object
grade          object
exam           object
score         float64
gender         object
age            object
first_name     object
last_name      object
dtype: object
10.620445344129555


# Missing Values
Missing values are a routine problem in datasets and can occur for many reasons: incomplete data collection, errors in storage, or simple human mistakes. In pandas, these show up as `NaN` (Not a Number), which represents the absence of a value. While some calculations can ignore `NaN`s, others will fail or give misleading results unless the gaps are handled properly.
There are two main strategies to deal with missing values:

**1. Dropping rows with missing data.**
If you don’t want incomplete rows to affect your analysis, you can remove them using `.dropna()`. By default, this deletes any row containing a `NaN`. For more targeted cleaning, you can specify a `subset`, which only removes rows where certain columns are missing. For instance, dropping rows with `NaN` in `num_guests` keeps the rest of the table intact.

**2. Filling in missing data.**
Sometimes it’s more useful to replace missing values rather than delete them, especially if losing rows would throw away important information. The `.fillna()` method allows you to substitute `NaN`s with chosen values. A common practice is to fill with the mean of the column or another statistic, so the filled values are consistent with the data’s distribution. For example, filling a missing `bill` with the average bill and missing `num_guests` with the average number of guests preserves the dataset size while making it usable for analysis.

Choosing between dropping and filling depends on the context: dropping is safer when you want only complete records, while filling is helpful when you need to retain all rows but ensure the data is valid for calculations. Both are essential techniques for preparing clean, reliable datasets.

In [11]:
# 1. Get the mean of the score column before filling NaNs
score_mean = students["score"].mean()
print(score_mean)

# 2. Replace all NaN values in the score column with 0
students["score"] = students["score"].fillna(0)

# 3. Get the mean of the score column again after filling
score_mean_2 = students["score"].mean()
print(score_mean_2)


77.69657422512235
72.30971659919028
